In [8]:
"""Imports and settings."""

%load_ext autoreload
%autoreload 2
# %flow mode reactive

import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objs as go
import seaborn as sns

import aeon
from aeon.schema.schemas import exp02, social02

pd.set_option("display.max_rows", 100)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
"""Set some constants."""

cm2px = 5  # 1 cm = 5 px
s2ms = 1000  # 1 s = 1000 ms
area_thresh = 320  # min threshold in pixels for joint-blob area
min_speed = 2  # cm/s min speed for fighting
max_speed = 7  # cm/s max speed for fighting

In [81]:
"""Load centroid blob data."""

root = Path("Z:/aeon/data/raw/AEON3/social0.2")
start, end = pd.Timestamp("2024-02-22 18:00:00"), pd.Timestamp("2024-02-22 19:00:00"),
centroid_blob_data = aeon.load(root, exp02.CameraTop.Position, start, end)

In [82]:
"""Drop nans."""

centroid_blob_data.dropna(inplace=True)

In [83]:
"""Get blob speed"""

dxy = centroid_blob_data[["x", "y"]].diff().values[1:]
dt = (np.diff(centroid_blob_data.index) / 1e6).astype(int)  # ms
centroid_blob_data["speed"] = np.concatenate(([0], np.linalg.norm(dxy, axis=1) / dt / cm2px * s2ms))  # cm/s
k = np.ones(10) / 10  # running avg filter kernel (10 frames)
centroid_blob_data["speed"] = np.convolve(centroid_blob_data["speed"], k, mode="same")
display(centroid_blob_data)

In [84]:
centroid_blob_data

,x,y,angle,major,minor,area,id,speed
time,,,,,,,,
2024-02-22 18:00:00.000000000,884.013977,53.274940,0.464139,32.068550,11.487731,274.0,0.0,17.375743
2024-02-22 18:00:00.019999981,890.113342,55.496655,0.398757,70.781845,9.892250,423.5,0.0,35.713902
2024-02-22 18:00:00.039999962,886.769287,54.306698,0.380861,73.145416,9.928225,425.5,0.0,39.434125
2024-02-22 18:00:00.059999943,883.370056,53.172722,0.358656,73.616127,9.987156,426.5,0.0,43.611349
2024-02-22 18:00:00.079999924,880.600464,52.373138,0.341265,76.066849,10.689726,448.0,0.0,47.639160
...,...,...,...,...,...,...,...,...
2024-02-22 18:59:59.920000076,1279.382080,503.560211,0.630973,22.082439,14.568158,263.0,0.0,1.514771
2024-02-22 18:59:59.940000057,1279.302612,503.659760,0.646705,22.191914,14.638694,266.0,0.0,1.223831
2024-02-22 18:59:59.960000038,1279.394287,503.558716,0.656961,22.308121,14.344864,262.5,0.0,1.018267


In [95]:
"""Get big blob data (when they're close together)"""

big_blobs = centroid_blob_data[centroid_blob_data["area"] > area_thresh]
# Filter by speed
big_blob_fightin = big_blobs[(big_blobs["speed"] > min_speed) & (big_blobs["speed"] < max_speed)]

In [97]:
display(big_blob_fightin.iloc[0:100])

,x,y,angle,major,minor,area,id,speed
time,,,,,,,,
2024-02-22 18:00:11.500000000,1227.493530,587.591492,-1.439926,50.618809,18.458868,474.5,0.0,5.362257
2024-02-22 18:00:11.519999981,1227.687134,587.798828,-1.428151,50.821526,18.311947,480.5,0.0,5.577104
2024-02-22 18:00:11.539999962,1228.124878,588.146301,-1.409051,51.638401,17.941162,474.0,0.0,4.781300
2024-02-22 18:00:11.559999943,1228.316284,588.777222,-1.396915,53.005066,18.651241,478.0,0.0,4.843798
2024-02-22 18:00:11.579999924,1228.162598,589.317688,-1.393139,53.833050,19.308222,490.0,0.0,5.221738
2024-02-22 18:00:11.599999905,1228.284912,589.815918,-1.379495,54.697922,18.567047,474.5,0.0,5.188085
2024-02-22 18:00:11.619999886,1228.304199,590.041931,-1.386789,55.410965,18.301546,469.0,0.0,5.010472
2024-02-22 18:00:12.239999771,1243.657349,539.769836,0.665816,29.064093,16.353760,376.5,0.0,4.220237
2024-02-22 18:00:12.260000229,1243.894165,540.074097,0.731258,29.446667,16.369587,378.0,0.0,3.819884
